In [4]:
import numpy as np
import random
import sys
import io


from improve import improve_algorithm
from helpers import extract_code, format_response, insert_print_statement, delete_print_statement

In [5]:
from print.chat_models.crfm_chat_llm import crfmChatLLM
llm = crfmChatLLM(model_name="openai/gpt-4-0613", crfm_api_key="")

ModuleNotFoundError: No module named 'print.chat_models'; 'print' is not a package

In [3]:
def utility_with_feedback(algorithm_str: str):
    "Evaluates the informativeness of print statements in an algorithm and returns the print feedback."
    
    # task: compute element-wise sum across two lists 
    list1 = [2, 3, 4, 2, 1]
    list2 = [5, 4, 3, 2, 1]
    expected_output = [7, 7, 7, 4, 2]
    
    # redirect stdout to capture print statements
    old_stdout = sys.stdout
    new_stdout = io.StringIO()
    sys.stdout = new_stdout
    
    result = None
    try:
        exec(algorithm_str, globals())
        result = algorithm(list1, list2)  # this will run the algorithm defined in algorithm_str
        score = np.sum([r == e for r, e in zip(result, expected_output)]) / len(expected_output) # this is the actual utility score, higher is better 
    except Exception as e:
        sys.stdout = old_stdout
        return (f"Error in execution: {e}", "")

    # Reset stdout and get feedback
    feedback = new_stdout.getvalue()
    sys.stdout = old_stdout
    
    return (score, feedback)

utility_str = """def utility_with_feedback(algorithm_str: str):
    "Evaluates the informativeness of print statements in an algorithm and returns the print feedback."
    
    # task: compute element-wise sum across two lists 
    list1 = [2, 3, 4, 2, 1]
    list2 = [5, 4, 3, 2, 1]
    expected_output = [7, 7, 7, 4, 2]
    
    # redirect stdout to capture print statements
    old_stdout = sys.stdout
    new_stdout = io.StringIO()
    sys.stdout = new_stdout
    
    result = None
    try:
        exec(algorithm_str, globals())
        result = algorithm(list1, list2)  # this will run the algorithm defined in algorithm_str
        score = np.sum([r == e for r, e in zip(result, expected_output)]) / len(expected_output) # this is the actual utility score, higher is better 
    except Exception as e:
        sys.stdout = old_stdout
        return (f"Error in execution: {{e}}", "")

    # Reset stdout and get feedback
    feedback = new_stdout.getvalue()
    sys.stdout = old_stdout
    
    return (score, feedback)"""

In [4]:
initial_solution = """
def algorithm(list1, list2):
    print(list1)
    result = []
    for i in range(len(list1)):
        sum_value = list1[i] + list2[i] * 2
        result.append(sum_value)
    return result"""

In [5]:
# Choose the print statement's location and content
line, variable = choose_print_statement(initial_solution, utility_str, llm)
print(f"Chosen Line: {line}, Variable: {variable}")

# Delete the existing print statement
initial_solution = delete_print_statement(initial_solution)

# Insert the new print statement
modified_solution = insert_print_statement(initial_solution, line, variable)
print(modified_solution)


# Evaluate the modified algorithm using the utility_with_feedback function
score, feedback = utility_with_feedback(modified_solution)
print(f"\nScore: {score}")
print(f"Feedback:\n{feedback}")

[{'role': 'system', 'content': 'Decide where and what to print in the given solution to help with debugging. You can only insert one print statement and only print one variable using the print statement.'}, {'role': 'user', 'content': 'Choose a new location and content (i.e., variable to print) for a print statement in the following solution to assist with debugging:\n\n```python\n\ndef algorithm(list1, list2):\n    print(list1)\n    result = []\n    for i in range(len(list1)):\n        sum_value = list1[i] + list2[i] * 2\n        result.append(sum_value)\n    return result\n```\n\nYou will use the return of the print statement to improve the above solution. Upon improvement, you will be evaluated based on the following utility:\n\n```python\ndef utility_with_feedback(algorithm_str: str):\n    "Evaluates the informativeness of print statements in an algorithm and returns the print feedback."\n    \n    # task: compute element-wise sum across two lists \n    list1 = [2, 3, 4, 2, 1]\n   

In [8]:
from lpn import utility_class, initial_solution

best_sol = improve_algorithm(initial_solution=initial_solution, utility=utility_class, language_model=llm)
print(best_sol)

[{'role': 'system', 'content': 'You are an expert computer science researcher and programmer, especially skilled at optimizing algorithms.'}, {'role': 'user', 'content': 'Improve the following solution:\n\n```python\nfrom sklearn.linear_model import LogisticRegression\n\ndef algorithm(train_samples, train_parity, test_samples):\n    return 100\n```\n\nYou will be evaluated based on this score function:\n\n```python\nimport random\nimport numpy as np\nimport time\n\ndef utility(algorithm_str: str):\n    "Implements the parity learning task. Returns the number of correct predictions."\n    \n    n_tests = 1\n    average_correct = 0\n\n    try:\n        exec(algorithm_str, globals())\n    except:\n        return 0\n    \n    for _ in range(n_tests):\n        start_time = time.time()\n        n_bits = 10\n        p_true = 0.3\n        n_train_samples = 100\n        n_test_samples = 20\n        noise_level = 0.05\n        true_bits = np.random.binomial(1, p_true, n_bits)\n\n        samples 

In [ ]:
algorithm_str = """def algorithm(train_samples, train_parity, test_samples):
    model = LogisticRegression()
    model.fit(train_samples, train_parity)
    predictions = model.predict(test_samples)
    return predictions"""

from lpn import utility_class

def meta_utility(improve_str: str):

    """
    Evaluates the algorithm in improve_str to improve the algorithm in algorithm_str, 
    according to some downstream utility function. This meta-utility function can only be called n times.
    """
    n_tests = 5
    expected_utility = 0
    for _ in range(n_tests):
        try: 
            exec(improve_str, globals())
        except:
            continue
        improved_algorithm_str = improve_algorithm(algorithm_str, utility_class, llm)
        expected_utility += utility_class.fun(improved_algorithm_str) / n_tests
    return expected_utility
